In [100]:
# import nisnap
import pyxnat
import xnat
from pyxnat import Interface
import pandas as pd
import nibabel as nib
from tqdm import tqdm


In [101]:
my_pyxnat = Interface(server="https://xnat.crmbm.univ-amu.fr",
                    user='hdary')

In [3]:
# if os.environ['XNAT_XSI_TYPE'] != 'xnat:projectData':
#     raise Exception('Must be started from an XNAT project.')

# project = os.environ['XNAT_ITEM_ID']

In [102]:
project = 'TERRA_BRAIN_EPINOV'

In [103]:
def get_resources(my_pyxnat, project):

    resources = []
    experiments = my_pyxnat.array.experiments(project_id=project, columns=['ID', 'project', 'label', 'subject_ID']).data
    # For each experiments fetch all the resources associated with it
    for exp in tqdm(experiments):
        j = my_pyxnat.get('{}/{}'.format(exp['URI'], 'resources')).json()
        exp_res = j['ResultSet']['Result']

        if exp_res:
            resources.extend([[exp['project'], exp['ID'], exp['label'], exp['subject_ID'],
                               r['xnat_abstractresource_id'],
                               r['label']] for r in exp_res])
        else:
            resources.append([exp['project'], exp['ID'],  exp['label'], exp['subject_ID'],
                              'No Data', 'No Data'])
    return resources

In [104]:
def get_subjects(my_pyxnat, project):
    dico = {i.id() :i.label() for i in my_pyxnat.select.project(project).subjects()}
    return dico

In [105]:
dico = get_subjects(my_pyxnat, project)

In [106]:
dico

{'XNAT_CRMBM_S00106': '01_55_L_M',
 'XNAT_CRMBM_S00107': '01_45_B-N',
 'XNAT_CRMBM_S00108': '01_85_G-S',
 'XNAT_CRMBM_S00109': '01-59-P-V',
 'XNAT_CRMBM_S00114': '01-68-P-N',
 'XNAT_CRMBM_S00125': '01-75-R-S',
 'XNAT_CRMBM_S00127': '01-63-B-K',
 'XNAT_CRMBM_S00110': '01_77_B_B',
 'XNAT_CRMBM_S00111': '01_86_P-L',
 'XNAT_CRMBM_S00116': '01-83-P-G',
 'XNAT_CRMBM_S00122': '01_71_D-C',
 'XNAT_CRMBM_S00126': '01_57_FC',
 'XNAT_CRMBM_S00113': '01_80_V_T',
 'XNAT_CRMBM_S00117': '01-64-B-A',
 'XNAT_CRMBM_S00115': '01-54-D-R',
 'XNAT_CRMBM_S00118': '01-61-B-L',
 'XNAT_CRMBM_S00119': '01-49-R-D',
 'XNAT_CRMBM_S00120': '01-70-L-E',
 'XNAT_CRMBM_S00123': '01-79-G-S',
 'XNAT_CRMBM_S00145': '01_72_M-V',
 'XNAT_CRMBM_S00146': '01_53_GL',
 'XNAT_CRMBM_S00147': '01_62_T-E',
 'XNAT_CRMBM_S00124': '01_69_CO',
 'XNAT_CRMBM_S00128': '01_47_Q-M',
 'XNAT_CRMBM_S00129': '01-56-C-D',
 'XNAT_CRMBM_S00130': '01_84_GM',
 'XNAT_CRMBM_S00131': '01-46_L-J',
 'XNAT_CRMBM_S00132': '01-78-M-C',
 'XNAT_CRMBM_S00133': '0

In [107]:
resources = get_resources(my_pyxnat, project)

  0%|          | 0/34 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:00<00:00, 67.46it/s]


In [108]:
dico_resources = {}
for r in resources:
    if r[5] == 'No Data':
        continue
    if r[5] in dico_resources.keys():
        dico_resources[r[5]].append(dico[r[3]])
    else:
        dico_resources[r[5]] = []
        dico_resources[r[5]].append(dico[r[3]])

In [109]:
# max_length = max(len(v) for v in dico_resources.values())

# # Ajouter des None aux listes pour qu'elles aient la même longueur
# for key in dico_resources:
#     dico_resources[key].extend([None] * (max_length - len(dico_resources[key])))


In [110]:
# Créer un ensemble contenant tous les sujets
sujets = set()
for valeurs in dico_resources.values():
    sujets.update(valeurs)

# Créer une copie du dictionnaire pour conserver les noms des sujets dans les valeurs
dictionnaire_modifie = {colonne: [sujet in valeurs for sujet in sujets] for colonne, valeurs in dico_resources.items()}

# Convertir le dictionnaire modifié en DataFrame
index = list(sujets)
dataframe = pd.DataFrame(dictionnaire_modifie)

dataframe.insert(0, "Sujets", list(sujets))

# Remplacer les index par les noms des sujets
# dataframe.index = list(sujets)


In [111]:
dictionnaire_modifie

{'FREESURFER': [True, True, True],
 'Atlas': [False, False, True],
 'FASTSURFER': [False, False, True],
 'GIF': [False, False, True]}

In [112]:
dico_resources

{'FREESURFER': ['01-59-P-V', '01-68-P-N', '01-66-L-Q'],
 'Atlas': ['01-59-P-V'],
 'FASTSURFER': ['01-59-P-V'],
 'GIF': ['01-59-P-V']}

In [113]:
dataframe

,Sujets,FREESURFER,Atlas,FASTSURFER,GIF
0,01-68-P-N,True,False,False,False
1,01-66-L-Q,True,False,False,False
2,01-59-P-V,True,True,True,True


In [114]:
import plotly.express as px
import plotly.graph_objects as go

In [52]:
cell_colors = [['white' if j == 0 else 'green' if val else 'red' for val in dataframe[col]] for j, col in enumerate(dataframe.columns)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(dataframe.columns)),
    cells=dict(values=[dataframe[col] for col in dataframe.columns],
               fill_color=cell_colors)
)])

# Afficher le tableau Plotly
fig.show()

In [53]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Générer un DataFrame de test
np.random.seed(1)
data = {
    'Colonne1': np.random.randn(100),
    'Colonne2': np.random.randn(100),
    'Colonne3': np.random.randn(100)
}
df = pd.DataFrame(data)

# Créer une figure
fig = go.Figure()

# Ajouter les traces pour chaque colonne
for col in df.columns:
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[col],
            mode="markers",
            name=col
        )
    )

# Créer une liste de formes pour chaque colonne
shapes = []
for i, col in enumerate(df.columns):
    shapes.append(
        dict(type="rect",
             xref="paper", yref="paper",
             x0=0, y0=0,
             x1=0, y1=1,
             fillcolor="rgba(0,0,0,0)",
             line=dict(color="rgba(0,0,0,0)", width=0),
             visible="legendonly",  # Initialement caché
             name=col)
    )

# Ajouter les formes à la figure
fig.update_layout(shapes=shapes)

# Créer des boutons pour chaque colonne
buttons = []
for col in df.columns:
    button = dict(label=col,
                  method="update",
                  args=[{"visible": [col == trace.name for trace in fig.data]},
                        {"shapes": [shape if shape['name'] == col else dict(shape, visible="legendonly") for shape in shapes]}])
    buttons.append(button)

# Ajouter un bouton pour afficher toutes les colonnes
buttons.append(dict(label="Toutes les colonnes",
                    method="update",
                    args=[{"visible": [True] * len(df.columns)},
                          {"shapes": shapes}]))

# Ajouter le menu déroulant
fig.update_layout(updatemenus=[dict(type="buttons",
                                    direction="down",
                                    buttons=buttons,
                                    x=1.1,
                                    y=1.05)])

# Afficher la figure
fig.show()


In [130]:
[dataframe[col] for col in dataframe.columns]

[0    01-68-P-N
 1    01-66-L-Q
 2    01-59-P-V
 Name: Sujets, dtype: object,
 0    True
 1    True
 2    True
 Name: FREESURFER, dtype: bool,
 0    False
 1    False
 2     True
 Name: Atlas, dtype: bool,
 0    False
 1    False
 2     True
 Name: FASTSURFER, dtype: bool,
 0    False
 1    False
 2     True
 Name: GIF, dtype: bool]

In [134]:
dataframe.index

RangeIndex(start=0, stop=3, step=1)

In [133]:
import pandas as pd
import plotly.graph_objects as go

cell_colors = [['white' if j == 0 else 'green' if val else 'red' for val in dataframe[col]] for j, col in enumerate(dataframe.columns)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(dataframe.columns)),
    cells=dict(values=[dataframe[col] for col in dataframe.columns],
               fill_color=cell_colors)
)])

# Afficher le tableau Plotly

# Créer des boutons pour afficher les colonnes correspondantes (à l'exception de la colonne "Sujets")
buttons = []
# for i, col in enumerate(df.columns[1:]):
#     button = dict(
#         label=col,
#         method="update",
#     #     args=[{"visible": [True if j == i+1 else False for j in range(len(df.columns))]},
#     #           {"title": f"Tableau avec la colonne {col}"}]
#     # )
#     # buttons.append(button)


#         # args=[{"cells.values": [[dataframe.iloc[j]["Sujets"]] + [dataframe.iloc[j][col] if k == i else None for k in range(len(dataframe.columns)-1)] for j in range(len(dataframe))]},
#         args=[{"cells.values": [[dataframe.iloc[row, 0]] + [dataframe.iloc[row, j] if j == i+1 else None for j in range(1, len(dataframe.columns))] for row in range(len(dataframe))]},

#         # args=[{"cells.values": [[row[0]] + [row[j+1] if j == i else None for j in range(len(dataframe.columns)-1)] for row in dataframe.values]},
#               {"title": f"Tableau avec la colonne {col}"}]
#     )
#     buttons.append(button)


# Ajouter un bouton "reset" pour réinitialiser le tableau
reset_button = dict(
    label="Reset",
    method="update",
    # args=[{"cells.values": [df.iloc[i].values.T for i in range(len(df))]},
    # args=[{"cells.values": [dataframe[col] for col in dataframe.index]},
    args=[{"cells.values": [dataframe.iloc[i].values for i in range(len(dataframe))]},
          {"title": "Tableau avec toutes les colonnes"}]
)
buttons.append(reset_button)

# Mettre à jour les boutons dans la mise en page de la figure
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ]
)

# Afficher le tableau Plotly
fig.show()


KeyError: 0

In [1]:
import pandas as pd
import plotly.graph_objects as go

# Votre DataFrame
data = {
    'FREESURFER': [True, False, True],
    'Atlas': [True, True, False],
    'FASTSURFER': [False, True, True]
}
index = ['01-59-P-V', '01-68-P-N', '01-66-L-Q']
subjects = index  # Utilisation de la liste des sujets comme première colonne
df = pd.DataFrame(data)

# Ajouter la liste des sujets comme première colonne du DataFrame
df.insert(0, "Sujets", subjects)

# Créer un tableau Plotly initial avec toutes les colonnes visibles
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns)),
    cells=dict(values=[df[col] for col in df.columns])
)])

# Créer des boutons pour afficher les colonnes correspondantes
buttons = []
for col in df.columns[1:]:
    visible_columns = ['Sujets', col]  # Colonnes visibles
    button = dict(
        label=col,
        method="update",
        args=[{"cells.values": [df[col] for col in df.columns]},

        args=[{"cells.values": [df.iloc[j].values.tolist() if i == j else [''] * len(df.columns) for j in range(len(df))]},
        # args=[{"cells.values": [df[col] for col in df.columns]},
        #       {"title": f"Tableau avec la colonne {col}"}]
    )
    buttons.append(button)

# Ajouter un bouton "reset" pour réinitialiser le tableau
reset_button = dict(
    label="Reset",
    method="update",
    args=[{"cells.values": [df.iloc[i].values.T for i in range(len(df))]},
          {"title": "Tableau avec toutes les colonnes"}]
)
buttons.append(reset_button)

# Mettre à jour les boutons dans la mise en page de la figure
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ]
)

# Afficher le tableau Plotly
fig.show()


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 32 (2370266399.py, line 35)

In [ ]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [86]:
dico_resources = {}
freesurfer_res_list = []
atlas_res_list = []

for sub in my_pyxnat.select.project(project).subjects():
    for exp in sub.experiments():
        for res in exp.resources():
            # if not os.path.exists()
            if res.label() == 'FREESURFER':
                freesurfer_res_list.append(res)
            if res.label() == 'Atlas':
                atlas_res_list.append(res)

In [87]:
freesurfer_list

NameError: name 'freesurfer_list' is not defined

In [94]:
freesurfer_res_list[0].aseg()

,measurement,region,value
0,Volume_mm3,BrainSegVol,1022580.374364
1,Volume_mm3,BrainSegVolNotVent,980107.281317
2,Volume_mm3,VentricleChoroidVol,36647.69615
3,Volume_mm3,lhCortexVol,204580.125985
4,Volume_mm3,rhCortexVol,213219.579237
...,...,...,...
419,normRange,CC_Posterior,98.0000
420,normRange,CC_Mid_Posterior,69.0000
421,normRange,CC_Central,96.0000
422,normRange,CC_Mid_Anterior,110.0000


In [98]:
filtre = freesurfer_res_list[0].aseg()["region"] == 'eTIV'
ligne_filtree = freesurfer_res_list[0].aseg()[filtre]
ligne_filtree['value'].iloc[0]

# freesurfer_res_list[0].aseg().at[row_index, region]


987811.52958

In [ ]:
freesurfer_list[0].aparc()

,side,measurement,region,value
0,None,Volume_mm3,BrainSegVol,1022580.374364
1,None,Volume_mm3,BrainSegVolNotVent,980107.281317
2,None,Volume_mm3,BrainSegVolNotVentSurf,980107.281317
3,None,Volume_mm3,CortexVol,417799.705223
4,None,Volume_mm3,SupraTentorialVol,907305.992304
...,...,...,...,...
627,right,CurvInd,supramarginal,6.3
628,right,CurvInd,frontalpole,0.8
629,right,CurvInd,temporalpole,1.0
630,right,CurvInd,transversetemporal,0.4
